In [1]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn import naive_bayes
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score

/Users/Stephen/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
social = pd.read_csv('social_data.csv')

In [3]:
housing = pd.read_csv('housing_data.csv')

In [4]:
economic = pd.read_csv('economic_data.csv')

In [5]:
acs1 = pd.read_csv('acs1_data.csv')

/Users/Stephen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (75,76,77,78,79,81,83,85,87,105,106,223,224,225,226,227,228,229,230,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,485,523,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,575) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
census = pd.read_csv('census_data.csv')

In [7]:
results = pd.read_csv('results_data.csv')

In [49]:
len(acs1)

1740

In [8]:
acs1 = pd.merge(acs1, results, how='right', on=['Year','District','State'])

In [51]:
len(acs1)

1773

In [9]:
sf1 = pd.merge(census, results, how='right',on = ['Year','District','State'])

In [10]:
social = pd.merge(social, results, how='right',on = ['Year','District','State'])
economic = pd.merge(economic, results, how='right',on = ['Year','District','State'])
housing = pd.merge(housing, results, how='right',on = ['Year','District','State'])

In [11]:
sf1 = sf1.dropna()
acs1 = acs1.dropna()
social = social.dropna()
economic = economic.dropna()
housing = housing.dropna()

In [12]:
sf1 = sf1.drop_duplicates()
acs1 = acs1.drop_duplicates()
social = social.drop_duplicates()
economic = economic.drop_duplicates()
housing = housing.drop_duplicates()

In [13]:
sf1_predictions = sf1['Party']

sf1_numeric_cols = []
for column in sf1.columns:
    try:
        sf1[column] = pd.to_numeric(sf1[column])
        sf1_numeric_cols.append(column)
    except:
        pass
    
sf1_num = sf1[sf1_numeric_cols]

In [14]:
acs1_predictions = acs1['Party']

acs1_numeric_cols = []
for column in acs1.columns:
    try:
        acs1[column] = pd.to_numeric(acs1[column])
        acs1_numeric_cols.append(column)
    except:
        pass
    
acs1_num = acs1[acs1_numeric_cols]

In [15]:
social_predictions = social['Party']

social_numeric_cols = []
for column in social.columns:
    try:
        social[column] = pd.to_numeric(social[column])
        social_numeric_cols.append(column)
    except:
        pass
    
social_num = social[social_numeric_cols]

In [16]:
economic_predictions = economic['Party']

economic_numeric_cols = []
for column in economic.columns:
    try:
        economic[column] = pd.to_numeric(economic[column])
        economic_numeric_cols.append(column)
    except:
        pass
    
economic_num = economic[economic_numeric_cols]

In [17]:
housing_predictions = housing['Party']

housing_numeric_cols = []
for column in housing.columns:
    try:
        housing[column] = pd.to_numeric(housing[column])
        housing_numeric_cols.append(column)
    except:
        pass
    
housing_num = housing[housing_numeric_cols]

In [18]:
sf1_num.drop(['Unnamed: 0_x', 'level_0','index','Id2','District','State','Year','Unnamed: 0_y'], axis=1, inplace=True)
acs1_num.drop(['Unnamed: 0_x','index_x','Id2_x','Unnamed: 0_y','index','Id2','Year','District','State','index_y','Id2_y'], axis=1, inplace=True)
social_num.drop(['Unnamed: 0_x','index','Id2','Year','District','State','Unnamed: 0_y'], axis=1, inplace=True)
economic_num.drop(['Unnamed: 0_x','index','Id2','Year','District','State','Unnamed: 0_y'], axis=1, inplace=True)
housing_num.drop(['Unnamed: 0_x','index','Id2','Year','District','State','Unnamed: 0_y'], axis=1, inplace=True)

/Users/Stephen/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
print(len(sf1_num),len(sf1_num.columns))
print(len(acs1_num),len(acs1_num.columns))
print(len(social_num),len(social_num.columns))
print(len(economic_num),len(economic_num.columns))
print(len(housing_num),len(housing_num.columns))

1747 361
1747 636
1747 170
1747 238
1747 228


In [20]:
sf1_numerical = []
sf1_percent = []

for column in sf1_num:
    if column[0] == 'N':
        sf1_numerical.append(column)
    elif column[0] == 'P':
        sf1_percent.append(column)
        
acs1_numerical = []
acs1_percent = []

for column in acs1_num:
    if column[0] == 'E':
        acs1_numerical.append(column)
    elif column[0] == 'P':
        acs1_percent.append(column)
        
social_numerical = []
social_percent = []

for column in social_num:
    if column[0] == 'E':
        social_numerical.append(column)
    elif column[0] == 'P':
        social_percent.append(column)
        
economic_numerical = []
economic_percent = []

for column in economic_num:
    if column[0] == 'E':
        economic_numerical.append(column)
    elif column[0] == 'P':
        economic_percent.append(column)
        
housing_numerical = []
housing_percent = []

for column in housing_num:
    if column[0] == 'E':
        housing_numerical.append(column)
    elif column[0] == 'P':
        housing_percent.append(column)

In [21]:
sf1_numbers = sf1_num[sf1_numerical]
sf1_percentages = sf1_num[sf1_percent]

acs1_numbers = acs1_num[acs1_numerical]
acs1_percentages = acs1_num[acs1_percent]

social_numbers = social_num[social_numerical]
social_percentages = social_num[social_percent]

economic_numbers = economic_num[economic_numerical]
economic_percentages = economic_num[economic_percent]

housing_numbers = housing_num[housing_numerical]
housing_percentages = housing_num[housing_percent]

In [22]:
sf1_predictions = pd.DataFrame(sf1_predictions)
sf1_predictions[['Democrat','Republican']] = pd.get_dummies(sf1_predictions['Party'])

acs1_predictions = pd.DataFrame(acs1_predictions)
acs1_predictions[['Democrat','Republican']] = pd.get_dummies(acs1_predictions['Party'])

social_predictions = pd.DataFrame(social_predictions)
social_predictions[['Democrat','Republican']] = pd.get_dummies(social_predictions['Party'])

economic_predictions = pd.DataFrame(economic_predictions)
economic_predictions[['Democrat','Republican']] = pd.get_dummies(economic_predictions['Party'])

housing_predictions = pd.DataFrame(housing_predictions)
housing_predictions[['Democrat','Republican']] = pd.get_dummies(housing_predictions['Party'])

In [23]:
predictions = sf1_predictions

In [67]:
sf1_numbers.to_csv('sf1_numbers.csv')
sf1_percentages.to_csv('sf1_percentages.csv')
acs1_numbers.to_csv('acs1_numbers.csv')
acs1_percentages.to_csv('acs1_percentages.csv')
social_numbers.to_csv('social_numbers.csv')
social_percentages.to_csv('social_percentages.csv')
economic_numbers.to_csv('economic_numbers.csv')
economic_percentages.to_csv('ec_percentages.csv')
sf1_numbers.to_csv('sf1_numbers.csv')
sf1_percentages.to_csv('sf1_percentages.csv')

predictions.to_csv('predictions.csv')

In [24]:
nbg = naive_bayes.GaussianNB()
nbb = naive_bayes.BernoulliNB()
nbm = naive_bayes.MultinomialNB()

In [25]:
dfs = [sf1_numbers, sf1_percentages, acs1_numbers, acs1_percentages, social_numbers, social_percentages, economic_numbers, economic_percentages, housing_numbers, housing_percentages]
names = ['sf1_numbers', 'sf1_percentages', 'acs1_numbers', 'acs1_percentages', 'social_numbers', 'social_percentages', 'economic_numbers', 'economic_percentages', 'housing_numbers', 'housing_percentages']

for index, df in enumerate(dfs):
    df.name = names[index]

In [26]:
len(predictions)

1747

In [28]:
y = predictions.loc[:, 'Democrat']

In [29]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    nbg.fit(X_train, y_train)
    print(df.name, 'Gaussian Cross Val Score:', np.mean(cross_val_score(nbg, X_train, y_train, scoring='roc_auc', cv=5)))
    print(df.name, 'Gaussian AUC Score:', roc_auc_score(y_test, nbg.predict(X_test)),'\n')
    
    nbb.fit(X_train, y_train)
    print(df.name, 'Bernoulli Cross Val Score:', np.mean(cross_val_score(nbb, X_train, y_train, scoring='roc_auc', cv=5)))
    print(df.name, 'Bernoulli AUC Score:', roc_auc_score(y_test, nbb.predict(X_test)),'\n')
    
    nbm.fit(X_train, y_train)
    print(df.name, 'Multinomial Cross Val Score:', np.mean(cross_val_score(nbm, X_train, y_train, scoring='roc_auc', cv=5)))
    print(df.name, 'Multinomial AUC Score:', roc_auc_score(y_test, nbm.predict(X_test)))

    print('\n')

sf1_numbers Gaussian Cross Val Score: 0.8358167413045827
sf1_numbers Gaussian AUC Score: 0.7397058823529412 

sf1_numbers Bernoulli Cross Val Score: 0.49923076923076926
sf1_numbers Bernoulli AUC Score: 0.5 

sf1_numbers Multinomial Cross Val Score: 0.7482934082513916
sf1_numbers Multinomial AUC Score: 0.7380718954248365


sf1_percentages Gaussian Cross Val Score: 0.839527149391891
sf1_percentages Gaussian AUC Score: 0.7398692810457517 

sf1_percentages Bernoulli Cross Val Score: 0.687159658706623
sf1_percentages Bernoulli AUC Score: 0.6684640522875818 

sf1_percentages Multinomial Cross Val Score: 0.7904993896899624
sf1_percentages Multinomial AUC Score: 0.7410130718954249


acs1_numbers Gaussian Cross Val Score: 0.8808268824562878
acs1_numbers Gaussian AUC Score: 0.7807189542483661 

acs1_numbers Bernoulli Cross Val Score: 0.5463581307304111
acs1_numbers Bernoulli AUC Score: 0.5859477124183007 

acs1_numbers Multinomial Cross Val Score: 0.7802700571890521
acs1_numbers Multinomial AUC 

In [30]:
dfs_numbers = [sf1_numbers, acs1_numbers, social_numbers, economic_numbers, housing_numbers]

scale = StandardScaler()

for df in dfs_numbers:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    X_train = scale.fit_transform(X_train)
    X_test = scale.transform(X_test)
    
    nbg.fit(X_train, y_train)
    print(df.name, 'Gaussian Cross Val Score:', np.mean(cross_val_score(nbg, X_train, y_train, scoring='roc_auc', cv=5)))
    print(df.name, 'Gaussian AUC Score:', roc_auc_score(y_test, nbg.predict(X_test)),'\n')
    
    nbb.fit(X_train, y_train)
    print(df.name, 'Bernoulli Cross Val Score:', np.mean(cross_val_score(nbb, X_train, y_train, scoring='roc_auc', cv=5)))
    print(df.name, 'Bernoulli AUC Score:', roc_auc_score(y_test, nbb.predict(X_test)),'\n')
    
    #nbm.fit(X_train, y_train)
    #print(df.name, 'Multinomial Cross Val Score:', np.mean(cross_val_score(nbm, X_train, y_train, scoring='roc_auc', cv=5)))
    #print(df.name, 'Multinomial AUC Score:', roc_auc_score(y_test, nbm.predict(X_test)))

    print('\n')


sf1_numbers Gaussian Cross Val Score: 0.8342870682306612
sf1_numbers Gaussian AUC Score: 0.7341503267973857 

sf1_numbers Bernoulli Cross Val Score: 0.8020781901205811
sf1_numbers Bernoulli AUC Score: 0.7428104575163399 



acs1_numbers Gaussian Cross Val Score: 0.8801891320790961
acs1_numbers Gaussian AUC Score: 0.7749999999999999 

acs1_numbers Bernoulli Cross Val Score: 0.8718650050225515
acs1_numbers Bernoulli AUC Score: 0.7856209150326797 



social_numbers Gaussian Cross Val Score: 0.885095155539014
social_numbers Gaussian AUC Score: 0.7715686274509804 

social_numbers Bernoulli Cross Val Score: 0.8709794255481785
social_numbers Bernoulli AUC Score: 0.7967320261437908 



economic_numbers Gaussian Cross Val Score: 0.8565551764296263
economic_numbers Gaussian AUC Score: 0.7521241830065359 

economic_numbers Bernoulli Cross Val Score: 0.8197247593777639
economic_numbers Bernoulli AUC Score: 0.7343137254901961 



housing_numbers Gaussian Cross Val Score: 0.8744594485998889
housing_

In [31]:
dfs = [sf1_numbers, sf1_percentages, acs1_numbers, acs1_percentages]

In [32]:
rf = RandomForestClassifier()

params = {'max_depth': range(1,21)}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [33]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    grid = GridSearchCV(rf, params, scoring = 'roc_auc', cv=kf)
    grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(grid, X_train, y_train, scoring='roc_auc', cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, grid.predict(X_test)))
    print(df.name, "Best depth:", grid.best_estimator_, '\n')

sf1_numbers Cross Val Score: 0.9610843596547045
sf1_numbers AUC Score: 0.9192810457516339
sf1_numbers Best depth: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False) 

sf1_percentages Cross Val Score: 0.9621971633589863
sf1_percentages AUC Score: 0.8926470588235295
sf1_percentages Best depth: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
          

In [34]:
dfs_numbers = ['sf1_numbers','acs1_numbers']

In [40]:
lr = LogisticRegression(penalty='l1')

params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(lr, params, scoring = 'roc_auc', cv=kf)

scale = StandardScaler()

In [41]:
for df in dfs:
    X = df.loc[:]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    if df.name in dfs_numbers:
        X_train = scale.fit_transform(X_train)
        X_test = scale.transform(X_test)

    grid.fit(X_train, y_train)

    print(df.name, "Cross Val Score:", np.mean(cross_val_score(lr, X_train, y_train, scoring='roc_auc', cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, grid.predict(X_test)))
    print(df.name, "Best C:", grid.best_estimator_, '\n')

sf1_numbers Cross Val Score: 0.9575756268184238
sf1_numbers AUC Score: 0.8764705882352941
sf1_numbers Best C: LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) 

sf1_percentages Cross Val Score: 0.954476028222337
sf1_percentages AUC Score: 0.9047385620915033
sf1_percentages Best C: LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) 

acs1_numbers Cross Val Score: 0.9551494183044987
acs1_numbers AUC Score: 0.8931372549019608
acs1_numbers Best C: LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, mult

In [ ]:
depths = [9,9,8,7]

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


#for index, df in enumerate(dfs):
    
X = sf1_numbers_m.loc[:]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

rf = RandomForestClassifier(max_depth=9)
model = rf.fit(X_train, y_train)

print("Cross Val Score:", np.mean(cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=kf)))
print("AUC Score:", roc_auc_score(y_test, model.predict(X_test)),'\n')

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

import_columns = {}

for f in range(10):
    import_columns[f] = X_train.columns[indices[f]]

pair_df = sf1_numbers_m[list(import_columns.values())]
pair_df.columns = range(0,10)

sns.pairplot(pair_df);

In [ ]:
X = sf1_numbers.loc[:]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

rf = RandomForestClassifier(max_depth=9)
rf.fit(X_train, y_train)

print("Cross Val Score:", np.mean(cross_val_score(rf, X_train, y_train, scoring='roc_auc', cv=kf)))
print(roc_auc_score(y_test, rf.predict(X_test)))
sns.heatmap(confusion_matrix(y_test, rf.predict(X_test)), annot=True);

In [ ]:
print(len(sf1_numbers_m.columns))
len(sf1_numbers.columns)

In [ ]:
sf1_numbers_m = sf1_numbers

In [ ]:
max_columns = import_columns.values()

In [ ]:
max_model = model
max_df = sf1_numbers_m

In [ ]:
sf1_max = sf1_numbers[list(max_columns)]

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = sf1_max.loc[:]
y = predictions.loc[:,'Democrat']
    
X = sf1_max.loc[:]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

rf = RandomForestClassifier(max_depth=9)
rf.fit(X_train, y_train)

print("Cross Val Score:", np.mean(cross_val_score(rf, X_train, y_train, scoring='roc_auc', cv=kf)))
print("AUC Score:", roc_auc_score(y_test, rf.predict(X_test)),'\n')

In [ ]:
sf1_numbers_m = sf1_numbers_m.drop(sf1_numbers[[import_columns[i] for i in [3,2,9]]], axis=1)

In [ ]:
sf1_numbers_m = sf1_numbers.copy()
sf1_percentages_m = sf1_percentages.copy()
acs1_numbers_m = acs1_numbers.copy()
acs1_percentages_m = acs1_percentages.copy()

In [ ]:
print(importances)
print(indices)
for f in range(10):
        print("{}. {} ({})".format(f+1, X_train.columns[indices[f]], importances[indices[f]]))

In [ ]:
cs = [10, 10, 1, 1, 10, 1, 1, 1, 1, 1]

for index, df in enumerate(dfs):
    
    X = df.loc[:]

    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    lr = LogisticRegression(C = cs[index])

    lr.fit(X_train, y_train)

    print(df.name, "Cross Val Score:", np.mean(cross_val_score(rf, X_train, y_train, scoring='roc_auc', cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, lr.predict(X_test)),'\n')

    coefs = lr.coef_

    abs_coefs = np.abs(coefs)
    indices = np.argsort(abs_coefs[0])[::-1]

    print("Top 10 Features\n")

    for f in range(10):
        print("{}. {} ({})".format(f+1, X.columns[indices[f]], coefs[0][indices[f]]))
    print('\n')

In [ ]:
sns.scatterplot('Percent; HEALTH INSURANCE COVERAGE - With health insurance coverage - With private health insurance',
            'Percent; HEALTH INSURANCE COVERAGE - With health insurance coverage - With public coverage',data=economic_percentages)

In [ ]:
pair = sf1_numbers[['Number; SEX AND AGE - Male population - Under 5 years', 'Number; SEX AND AGE - Female population - 10 to 14 years',
                   'Number; SEX AND AGE - Female population - Under 5 years']]

pair.columns = ['Male < 5','Female 10-14','Female < 5']
plt.figure(figsize=(100,100))
sns.pairplot(pair)

In [ ]:
pair = sf1_percentages[['Percent; SEX AND AGE - Female population - 20 to 24 years', 'Percent; SEX AND AGE - Female population - 21 years and over',
                   'Percent; SEX AND AGE - Total population - 20 to 24 years']]

pair.columns = ['Female < 20-24','Female > 21','Total 20-24']
sns.pairplot(pair);

In [ ]:
svm = SVC()

params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'kernel': ['linear','rbf','poly']}

kf = StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [ ]:
for df in dfs:
    X = df.loc[:]
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    scale = StandardScaler()
    
    X_train = scale.fit_transform(X_train)
    X_test = scale.transform(X_test)
    
    grid = GridSearchCV(svm, params, scoring = 'roc_auc', cv=kf)
    grid.fit(X_train, y_train)
    
    print(df.name, "Cross Val Score:", np.mean(cross_val_score(grid, X_train, y_train, scoring='roc_auc', cv=kf)))
    print(df.name, "AUC Score:", roc_auc_score(y_test, grid.predict(X_test)))
    print(df.name, "Best model:", grid.best_estimator_, '\n')

In [ ]:
dfs = [sf1_numbers, sf1_percentages, acs1_numbers, acs1_percentages, social_numbers, social_percentages, economic_numbers, economic_percentages,
      housing_numbers, housing_percentages]

for df in dfs:
    df.drop(['Democrat'], axis=1, inplace=True)

In [ ]:
sf1_numbers_with_predictions = sf1_numbers
sf1_numbers_with_predictions['Democrat'] = predictions.Democrat

sf1_percentages_with_predictions = sf1_percentages
sf1_percentages_with_predictions['Democrat'] = predictions.Democrat

acs1_numbers_with_predictions = acs1_numbers
acs1_numbers_with_predictions['Democrat'] = predictions.Democrat

acs1_percentages_with_predictions = acs1_percentages
acs1_percentages_with_predictions['Democrat'] = predictions.Democrat

In [ ]:
sf1_numbers_with_prediction

In [ ]:
sf1_numbers_corr = sf1_numbers_with_predictions.corr()
sf1_percentages_corr = sf1_percentages_with_predictions.corr()
acs1_numbers_corr = acs1_numbers_with_predictions.corr()
acs1_percentages_corr = acs1_percentages_with_predictions.corr()


dfs_corr = [sf1_numbers_corr,sf1_percentages_corr,acs1_numbers_corr,acs1_percentages_corr]

In [ ]:
len(sf1_numbers_with_predictions.columns)

In [ ]:
df = pd.DataFrame()

df['Columns'] = sf1_numbers.columns
df['Rows'] = sf1_numbers.columns[0]

df['corr'] = sf1_numbers_corr.lookup(df['Rows'], df['Columns'])
df['row_predicted'] = df['Rows'].corr(predictions['Democrat'])

In [ ]:
df

In [ ]:
def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

sf1_numbers_dropped = calculate_vif_(sf1_numbers, thresh=5.0)

In [ ]:
X = sf1_numbers_dropped.loc[:,:]
y = predictions.loc[:,'Democrat']

rf = RandomForestClassifier()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

rf.fit(X_train, y_train)
roc_auc_score(y_test, rf.predict(X_test))

In [ ]:
sf1_numbers_dropped = df.loc[:]
    
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

grid = GridSearchCV(rf, params, scoring = 'roc_auc', cv=kf)
grid.fit(X_train, y_train)

print(df.name, "Cross Val Score:", np.mean(cross_val_score(grid, X_train, y_train, scoring='roc_auc', cv=kf)))
print(df.name, "AUC Score:", roc_auc_score(y_test, grid.predict(X_test)))
print(df.name, "Best depth:", grid.best_estimator_, '\n')

In [ ]:
lr = LogisticRegression()

params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
sf1_numbers_dropped = df.loc[:]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scale = StandardScaler()

X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

grid = GridSearchCV(lr, params, scoring = 'roc_auc', cv=kf)
grid.fit(X_train, y_train)

print(df.name, "Cross Val Score:", np.mean(cross_val_score(grid, X_train, y_train, scoring='roc_auc', cv=kf)))
print(df.name, "AUC Score:", roc_auc_score(y_test, grid.predict(X_test)))
print(df.name, "Best depth:", grid.best_estimator_, '\n')

In [ ]:
rf = RandomForestClassifier()

params = {'max_depth': range(1,21)}

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
importances = rf_best.feature_importances_
    std = np.std([tree.feature_importances_ for tree in rf_best.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    
    print("Top 10 Features")
    
    for f in range(10):
        print("{}. {} ({})".format(f+1, X_train.columns[indices[f]], importances[indices[f]]))
    
    print('\n')

In [ ]:
lr = LogisticRegression()

In [ ]:
for df in dfs

In [ ]:
len(acs1)

In [ ]:
len(sf1)

In [ ]:
len(acs1.columns)

In [ ]:
sf1_predictions = sf1['Party']

In [ ]:
sf1_numeric_cols = []
for column in sf1.columns:
    try:
        sf1[column] = pd.to_numeric(sf1[column])
        sf1_numeric_cols.append(column)
    except:
        pass

In [ ]:
acs1_numeric_cols = []
for column in acs1.columns:
    try:
        acs1[column] = pd.to_numeric(acs1[column])
        acs1_numeric_cols.append(column)
    except:
        pass

In [ ]:
sf1_num = sf1[sf1_numeric_cols]

In [ ]:
acs1_num = acs1[acs1_numeric_cols]

In [ ]:
sf1_num.drop(['Unnamed: 0_x', 'level_0','index','Id2','District','State','Year','Unnamed: 0_y'], axis=1, inplace=True)

In [ ]:
len(acs1_num.columns)

In [ ]:
acs1_num.drop(['Unnamed: 0_x','index_x','Id2_x','Unnamed: 0_y'], axis=1, inplace=True)

In [ ]:
acs1_predictions = acs1['Party']

In [ ]:
acs1_predictions = pd.DataFrame(acs1_predictions)

In [ ]:
acs1_predictions[['Democrat','Republican']] = pd.get_dummies(acs1_predictions['Party'])

In [ ]:
sf1_predictions = pd.DataFrame(sf1_predictions)

In [ ]:
sf1_predictions[['Democrat','Republican']] = pd.get_dummies(sf1_predictions['Party'])

In [ ]:
X = acs1_num.iloc[:]
y = acs1_predictions.loc[:,'Democrat']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
nb = naive_bayes.MultinomialNB()
nb.fit(X_train, y_train)

output = classification_report(y_test, nb.predict(X_test))

In [ ]:
acs1_num.columns

In [ ]:
sf1_predictions = pd.DataFrame(sf1_predictions)

In [ ]:
sf1_predictions[['Democrat','Republican']] = pd.get_dummies(sf1_predictions['Party'])

In [ ]:
acs1_numerical = []
acs1_percent = []

for column in acs1_num:
    if column[0] == 'E':
        acs1_numerical.append(column)
    elif column[0] == 'P':
        acs1_percent.append(column)
        
sf1_numerical = []
sf1_percent = []

for column in sf1_num:
    if column[0] == 'N':
        sf1_numerical.append(column)
    elif column[0] == 'P':
        sf1_percent.append(column)

In [ ]:
len(sf1_percent)

In [ ]:
sf1_num_e = sf1_num[sf1_numerical]
sf1_num_p = sf1_num[sf1_percent]

In [ ]:
acs1_num_e = acs1_num[acs1_numerical]
acs1_num_p = acs1_num[acs1_percent]

In [ ]:
forest = RandomForestClassifier()

In [ ]:
grid = GridSearchCV(forest, [{'max_depth':list(range(1,21))}])

In [ ]:
X = acs1_num_e.iloc[:]
y = acs1_predictions['Democrat']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

nb.fit(X_train, y_train)

roc_auc_score(y_test, nb.predict(X_test))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

mod = nb.fit(X_train, y_train)

In [ ]:
with open('model.pickle', 'wb') as to_pickle:
    pickle.dump(mod, to_pickle)

In [ ]:
grid.fit(X_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
roc_auc_score(y_test, grid.predict(X_test))

In [ ]:
lr = LogisticRegression(penalty='l1')

lr.fit(X_train, y_train)

roc_auc_score(y_test, lr.predict(X_test))

In [ ]:
lr.coef_

In [ ]:
forest.fit(X_train, y_train)

importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,100))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")

plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.ylim([-1, X_train.shape[1]])
plt.show()


In [ ]:
importances = gr.feature_importances_
std = np.std([tree.feature_importances_ for tree in gr.estimators_], axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,100))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")

plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.ylim([-1, X_train.shape[1]])
plt.show()

In [ ]:
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print(X_train.columns[indices[f]])

In [ ]:
X = acs1_num_p.iloc[:]
y = acs1_predictions['Democrat']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

nb.fit(X_train, y_train)
print(roc_auc_score(y_train, nb.predict(X_train)))
roc_auc_score(y_test, nb.predict(X_test))

In [ ]:
X = sf1_num_p.iloc[:]
y = sf1_predictions['Democrat']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

nb.fit(X_train, y_train)

roc_auc_score(y_test, nb.predict(X_test))

In [ ]:
forest.fit(X_train, y_train)

importances = forest.feature_importances_
importances

std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis = 0)

indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,100))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")

plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.ylim([-1, X_train.shape[1]])
plt.show()

In [ ]:
X = sf1_num_e.iloc[:]
y = sf1_predictions['Democrat']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

nb.fit(X_train, y_train)

roc_auc_score(y_test, nb.predict(X_test))

In [ ]:
forest.fit(X_train, y_train)

importances = forest.feature_importances_
importances

std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis = 0)

indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,100))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")

plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.ylim([-1, X_train.shape[1]])
plt.show()

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
importances = forest.feature_importances_
importances

In [ ]:
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis = 0)

indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X_train.columns[indices[f]], X_train.columns[indices[f]]))



In [ ]:
plt.figure(figsize=(10,100))
plt.title("Feature importances")
plt.barh(range(X_train.shape[1]), importances[indices],
       color="r", xerr=std[indices], align="center")

plt.yticks(range(X_train.shape[1]), [X_train.columns[i] for i in indices])
plt.ylim([-1, X_train.shape[1]])
plt.show()